In [22]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
pytrends = TrendReq(hl='en-US', tz=360, timeout=(10,25))

In [24]:
def getdata(term):
	kw_list = [term]
	pytrends.build_payload(kw_list, cat=0, timeframe='today 5-y', geo='', gprop='')
	df = pytrends.interest_over_time()
	return df

In [25]:
def plotdata(term):
    df = getdata(term)
    df.plot()

In [26]:
stocks = pd.read_csv("data/daily_stocks.csv")

/home/clearly/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
newstocks = stocks[stocks['date']>20150000] # Restrict to data after 2015

In [37]:
newstocks.head()

,PERMNO,date,TICKER,BIDLO,ASKHI,PRC,BID,ASK,OPENPRC,RETX
2014,10026,20150102,JJSF,106.810,109.446,107.69,107.45,107.69,109.18,-0.009929
2015,10026,20150105,JJSF,106.760,108.430,107.24,107.25,107.39,107.41,-0.004179
2016,10026,20150106,JJSF,106.185,108.690,107.27,107.25,107.55,107.84,0.000280
2017,10026,20150107,JJSF,106.495,108.770,108.73,108.62,108.75,107.70,0.013611
2018,10026,20150108,JJSF,109.110,111.280,110.32,110.31,110.71,109.48,0.014623


In [38]:
def find_date_string(row):
    ds = str(row['date']-1)
    dates = ds[:4] + "-" + ds[4:6] + "-" + ds[6:]
    return dates

In [39]:
def getdf(searches, ticker):
    df = newstocks[newstocks["TICKER"]==ticker]
    df['date_string'] = df.apply(lambda row: find_date_string(row), axis=1)
    trends_data = getdata(searches[0])
    for i in range(1, len(searches)):
        s = searches[i]
        newtrends = getdata(s)
        trends_data = trends_data.join(newtrends, rsuffix='_'+s)
    s = set(trends_data.index.values)
    s = set([str(x)[:10] for x in s])
    df = df[df['date_string'].isin(s)]
    df.plot(y='BID')
    for s in searches:
        df[s] = df.apply(lambda row: trends_data.loc[row['date_string']][s], axis=1)
        df.plot(y=s)
    print(df)
    return df

In [41]:
from sklearn import linear_model

In [55]:
def easy(ticker):
    features = ["buy " + ticker, "sell " + ticker, ticker]
    data = getdf(features, ticker)
    X = pd.DataFrame(data, columns=features)
    y = data['BID']
    print("------------------------------------")
    print("x: ")
    print(X)
    print("------------------------------------")
    print("y: ")
    print(y)
    clf =  linear_model.SGDClassifier(alpha=0.0001, average=False, class_weight=None, early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5, random_state=None, shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)
    clf.fit(X, y[1])
    print(clf.score(X, y[1]))
    

In [56]:
easy("GOOGL")

/home/clearly/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
         PERMNO      date TICKER       BIDLO       ASKHI         PRC  \
8125309   90319  20150105  GOOGL   517.75000   527.98993   519.46002   
8125314   90319  20150112  GOOGL   490.91000   500.28000   497.06000   
8125323   90319  20150126  GOOGL   532.07001   541.50000   536.71997   
8125328   90319  20150202  GOOGL   521.71997   536.50000   532.20001   
8125333   90319  20150209  GOOGL   527.54999   533.88000   529.28003   
...         ...       ...    ...         ...         ...         ...   
8126295   90319  20181203  GOOGL  1111.72998  1135.00

ValueError: Unknown label type: (array([ 497.04999,  519.22998,  529.25   ,  531.76001,  534.90997,
        536.71002,  541.25   ,  542.98999,  543.47998,  543.85999,
        544.40997,  545.63   ,  545.77002,  546.41998,  548.53003,
        552.78003,  559.67999,  561.04999,  561.54999,  565.35999,
        566.09003,  571.71997,  573.81   ,  575.01001,  618.09998,
        624.12   ,  647.82001,  652.14001,  658.40002,  662.94   ,
        664.71997,  667.01001,  671.15997,  675.96997,  681.06   ,
        692.09998,  694.10999,  699.65002,  703.78003,  706.13   ,
        712.77002,  714.31   ,  717.23999,  727.03998,  728.94   ,
        729.13   ,  730.     ,  730.28998,  731.16998,  731.85999,
        733.06   ,  733.60999,  742.09003,  747.28998,  750.06   ,
        750.41998,  753.20001,  753.21002,  753.27002,  754.53998,
        757.27002,  757.53003,  759.40002,  760.44   ,  761.96997,
        762.53998,  762.84003,  764.70001,  772.69   ,  776.5    ,
        778.21002,  782.21002

TypeError: getdf() missing 1 required positional argument: 'ticker'

In [0]:
getdf("TSLA")

In [0]:
d = getdata('MSFT')

,MSFT,isPartial
date,,
2014-12-07,18,False
2014-12-14,18,False
2014-12-21,15,False
2014-12-28,14,False
2015-01-04,16,False
...,...,...
2019-10-27,74,False
2019-11-03,52,False
2019-11-10,55,False
